# Imports

In [3]:
import sys
import logging
import shutil
from uuid import uuid4
from tempfile import gettempdir
from os.path import join, isdir
import folium
from aequilibrae import Parameters, logger, Project

In [4]:
from functions.country_borders import get_country_borders
from functions.map_path import map_single_path
from functions.map_modes import map_all_modes

ModuleNotFoundError: No module named 'country_borders'

# Model place (Country) to build the model for

In [ ]:
model_place = 'Andorra'

# Environment setup

In [ ]:
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

In [ ]:
fldr = f'../{model_place}'

if isdir(fldr):
  shutil.rmtree(fldr)

# Create Road Network from OSM

In [ ]:
project = Project()
project.new(fldr)

In [ ]:
new_fields = [{'bridge': {'description': 'bridge flag', 'osm_source': 'bridge', 'type': 'text'}},
              {'tunnel': {'description': 'tunnel flag', 'osm_source': 'tunnel', 'type': 'text'}},
              {'toll': {'description': 'toll flag', 'osm_source': 'toll', 'type': 'text'}},
              {'surface': {'description': 'pavement surface', 'osm_source': 'surface', 'type': 'text'}}]

par = Parameters()
par.parameters['network']['links']['fields']['one-way'].extend(new_fields)
par.write_back()

In [ ]:
%%time
project.network.create_from_osm(place_name=model_place)

# Restricts import to the country's borders only

In [ ]:
place_geo = get_country_borders(model_place)

In [ ]:
place_geo

In [ ]:
project.conn.execute('CREATE TABLE IF NOT EXISTS country_borders("country_name" TEXT);')
project.conn.execute("SELECT AddGeometryColumn( 'country_borders', 'geometry', 4326, 'MULTIPOLYGON', 'XY' );")

project.conn.execute("SELECT CreateSpatialIndex( 'country_borders' , 'geometry' );")
project.conn.commit()
project.conn.execute('INSERT into country_borders(country_name, geometry) VALUES(?, GeomFromWKB(?, 4326));', [model_place, place_geo.wkb])
project.conn.commit()

In [ ]:
%%time
sql = """DELETE from Links where link_id not in (SELECT a.link_id
FROM links AS a, country_borders as b
WHERE ST_Intersects(a.geometry, b.geometry) = 1)"""

project.conn.execute(sql)
project.conn.commit()

# Displays the network

In [ ]:
map_all_modes(project)

# We test how would path computation work

In [ ]:
from aequilibrae.paths import PathResults

In [ ]:
project.conn.execute('update nodes set is_centroid=1')
project.conn.commit()

project.network.build_graphs()
graph = project.network.graphs['c']

# we also see what graphs are available
# project.network.graphs.keys()

# let's say we want to minimize distance
graph.set_graph('distance')

# And will skim time and distance while we are at it
graph.set_skimming(['distance'])

graph.set_blocked_centroid_flows(False)

In [ ]:
res = PathResults()
res.prepare(graph)

# compute a path from node 8 to 13
res.compute_path(3673, 6817)

In [ ]:
map_single_path(project, res)

In [ ]:
project.close()